In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [5]:
train_data,v_data,test_data=tfds.load(name="imdb_reviews",
                                      split=('train[:60%]','train[60%:]','test'),
                                      as_supervised=True)

In [6]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [7]:
train_ex_batch,train_lab_batch=next(iter(train_data.batch(10)))

In [8]:
train_ex_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [9]:
train_lab_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [11]:
em="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [14]:
hub_layer=hub.KerasLayer(em,input_shape=[],dtype=tf.string,trainable=True)

In [16]:
hub_layer(train_ex_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [18]:
model=tf.keras.Sequential()
model.add(hub_layer)

In [23]:
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [28]:
history=model.fit(train_data.shuffle(10000).batch(100),epochs=25,
                 validation_data=v_data.batch(100),verbose=1)

Epoch 1/25
150/150 [==============================] - 4s 20ms/step - loss: 0.7377 - accuracy: 0.5641 - val_loss: 0.6009 - val_accuracy: 0.6559
Epoch 2/25
150/150 [==============================] - 3s 20ms/step - loss: 0.5439 - accuracy: 0.7068 - val_loss: 0.5001 - val_accuracy: 0.7448
Epoch 3/25
150/150 [==============================] - 3s 18ms/step - loss: 0.4345 - accuracy: 0.7958 - val_loss: 0.4126 - val_accuracy: 0.8056
Epoch 4/25
150/150 [==============================] - 3s 19ms/step - loss: 0.3469 - accuracy: 0.8479 - val_loss: 0.3585 - val_accuracy: 0.8431
Epoch 5/25
150/150 [==============================] - 3s 21ms/step - loss: 0.2805 - accuracy: 0.8817 - val_loss: 0.3283 - val_accuracy: 0.8580
Epoch 6/25
150/150 [==============================] - 3s 19ms/step - loss: 0.2346 - accuracy: 0.9072 - val_loss: 0.3173 - val_accuracy: 0.8529
Epoch 7/25
150/150 [==============================] - 3s 17ms/step - loss: 0.2011 - accuracy: 0.9221 - val_loss: 0.3130 - val_accuracy: 0.8574

In [31]:
result=model.evaluate(test_data.batch(100),verbose=2)
for name,values in zip(model.metrics_names,result):
    print(name,values)

250/250 - 1s - loss: 0.6210 - accuracy: 0.8406 - 1s/epoch - 4ms/step
loss 0.6209637522697449
accuracy 0.8405600190162659
